In [ ]:
import numpy as np
import os
import sys
sys.path.append('/scratch/ulg/matnan/slongo/my_scripts/')
from utils import path


def make_md(mode='interactive', fpath=None):
    if mode not in ['interactive', 'from_file']:
        raise ValueError('The parameter "mode" must be either "interactive" or "from_file"')
    if mode == 'from_file' and fpath == None:
        raise ValueError('When "mode" = "from_file" a file path must be passed as "fpath"')

    def ask_input():
        input_pars = dict()
        input_pars['wdir'] = input("Where?")
        time = []
        time.append(input("How much time to request for NPT? (HH:MM:SS)"))
        ncores = []
        ncores.append(input("How many cores for NPT?"))
        nsteps = []
        nsteps.append(input("How many timesteps for NPT?")) 
        nsteps.append(input("How many timesteps for NVT?"))
        loginterval = []
        loginterval.append(input("Loginterval for NPT?"))
        nthrow = []
        nthrow.append(input("How many initial timesteps NOT to include in the trajectory for NPT (ntrhow)?"))   
        nthrow.append(input("How many initial timesteps NOT to include in the trajectory for NVT? (nthrow)"))
        input_pars['ninstances'] = input("How many NPT instances?")   
        dt.append(input("Duration of a timestep (in fs) for NPT:"))     
        input_pars['bool_nvt'] = input("Do you want the NVT MD runs to have the same setting (apart from nthrow, and nsteps, already asked separately) (yes/no)")    
        if bool_nvt == 'yes':
            time.append(time[0])
            ncores.append(ncores[0])       
            loginterval.append(loginterval[0])      
            dt.append(dt[0])   
        else:
            time.append(input("How much time to request for NVT? (HH:MM:SS)"))       
            ncores.append(input("How many cores for NVT?"))       
            loginterval.append(input("Loginterval for NVT?"))
            dt.append(input("Duration of a timestep (in fs) for NPT:"))

        bool_iso = input("Should isotropy be enforced on the system during NPT (yes/no)?")
        if bool_iso == 'yes'
            input_pars['iso'] = True       
        else:
            input_pars['iso'] = False

        bool_refine = input("Should the average NPT cell be refined before being used in the NVT (yes/no)?")
        if bool_refine == 'yes':
            input_pars['refine'] = True       
        else:
            input_pars['refine'] = False

        input_pars['time'] = time

        input_pars['matrix'] = input("Give the multiplication matrix for the supecell. Give all the nine elements 11, 12, 13, 21, 22, 23, 31, 32, 33")
        input_pars['mode'] = input("Choose mode\n1 - Homogeneous custom range\n2 - Custom list of temperatures")

        if str(mode) == '1':
            input_pars['T1'] = input("T start (K); integer:")
            input_pars['T2'] = input("T stop (K); integer (excluded):")
            input_pars['step'] = input("step (K); integer:")
        elif str(mode) == '2':
            ntemps = input("How many temperatures?")
            temps = []
            for i in range(ntemps):
                temp = input(f"Give the temperature n. {i+1}")
                temps.append(temp)
            input_pars['temps'] = temps
        else:
            print("You are not the smart one in your  family, are you?")
        input_pars = convert_input_pars(input_pars)
        return input_pars

    def get_input_from_file(filepath):
        filepath = Path(filepath)
        with open(filepath, 'r') as fl:
            lines = fl.readlines()
        input_pars = dict()
        for line in lines:
            var_name = line.split()[0]
            var_content = " ".join([str(x) for x in line.split()[1:]])
            input_pars[var_name] = var_content
        input_pars['iso'] = eval(input_pars['iso'])
        input_pars['refine'] = eval(input_pars['refine'])
        input_pars = convert_input_pars(input_pars)
        return input_pars

    def convert_input_pars(input_pars):
        input_pars['wdir'] = Path(input_pars['wdir'])
        input_pars['time'] = [str(x) for x in input_pars['time'].split()] 
        input_pars['ncores'] = [int(x) for x in input_pars['ncores_npt'].split()]
        input_pars['nsteps'] = [int(x) for x in input_pars['nsteps_npt'].split()]
        input_pars['loginterval'] = [int(x) for x in input_pars[''].split()]
        input_pars['nthrow'] = [int(x) for x in input_pars['npt_nthrow'].split()]
        input_pars['ninstances'] = int(input_pars['ninstances'])
        input_pars['dt'] = [float(x) for x in input_pars['dt_npt'].split()]
        mat_elems = input_pars['matrix'].split()
        matrix = [[mat_elems[0], mat_elems[1], mat_elems[2]],
                  [mat_elems[3], mat_elems[4], mat_elems[5]],
                  [mat_elems[6], mat_elems[7], mat_elems[8]]]
        input_pars['matrix'] = np.array(matrix, dtype='float')
        input_pars['mode'] = str(input_pars['mode'])

        if input_pars['mode'] == 1:
            input_pars['T1'] = float(input_pars['T1'])
            input_pars['T2'] = float(input_pars['T2'])
        else:
            input_pars['temps'] = np.array(input_pars['temps'], dtype='float')

        return input_pars


    scripts_dir = f'{root_dir}scripts_md/'
    os.system(f'mkdir -p {scripts_dir}')
    file_dir = path('/scratch/users/s/l/slongo/my_scripts/new_scripts_MD/scripts_to_copy')
    os.system(f'cp {file_dir}* {scripts_dir}')

    if mode == 'interactive':
        input_pars = ask_input()
    elif mode == 'from_file':
        input_pars = get_input_from_file(filepath)


    matrix = input_pars['matrix']

    for i, file in enumerate(['RunNPT_instance.py', 'RunNVT.py']):
        tokens = np.zeros(10)
        filepath = f'{scripts_dir}{file}'
        newlines = []
        with open(filepath, 'r') as fl:
            lines = fl.readlines()
        for line in lines:
            if 'rootdir = ' in line:
                newlines.append(f'rootdir = \'{root_dir}\'\n')
            elif 'nproc' in line and tokens[1] == 0:
                newlines.append(f"nproc = {input_pars['ncores'][i]}\n")
                tokens[1] = 1
            elif 'nsteps' in line and tokens[2] == 0:
                newlines.append(f"nsteps = {input_pars['nsteps'][i]}\n")
                tokens[2] = 1
            elif 'loginterval' in line and tokens[3] == 0:
                newlines.append(f"loginterval = {input_pars['loginterval'][i]}\n")
                tokens[3] = 1
            elif 'nthrow' in line and tokens[4] == 0:
                newlines.append(f"nthrow = {input_pars['nthrow'][i]}\n")
                tokens[4] = 1
            elif 'dt' in line and tokens[5] == 0:
                newlines.append(f"dt = {input_pars['dt[i]']}\n")
                tokens[5] = 1
            elif 'iso' in line and tokens[6] == 0:
                newlines.append(f"iso = {input_pars['iso']}\n")
                tokens[6] = 1
            elif 'mult_mat' in line and tokens[7] == 0:
                newlines.append(f"mult_mat = np.array([[{matrix[0,0]}, {matrix[0,1]}, {matrix[0,2]}], [{matrix[1,0]}, {matrix[1,1]}, {matrix[1,2]}], [{matrix[2,0]}, {matrix[2,1]}, {matrix[2,2]}]])\n")
                tokens[7] = 1
            elif 'refine =' and tokens[8] == 0:
                newlines.append(f"refine = {str(input_pars['refine'])}\n")
                tokens[8] = 1
            else:
                newlines.append(line)
        with open(filepath, 'w') as fl:
            fl.writelines(newlines)

    for i, file in enumerate(['npt_job.sh', 'nvt_job.sh']):
        filepath = f'{scripts_dir}{file}'
        newlines = []
        with open(filepath, 'r') as fl:
            lines = fl.readlines()
        for line in lines:
            if '#SBATCH --ntasks=' in line:
                newlines.append(f"#SBATCH --ntasks={input_pars['ncores'][i]}\n")
            elif '#SBATCH --time=' in line:
                newlines.append(f"#SBATCH --time={input_pars['time'][i]}\n")
            else:
                newlines.append(line)
        with open(filepath, 'w') as fl:
            fl.writelines(newlines)

    filepath = f'{scripts_dir}LaunchNPT_instances.py'
    newlines = []
    with open(filepath) as fl:
        lines = fl.readlines()
    for line in lines:
        if 'rootdir =' in line:
            newlines.append(f'rootdir = \'{root_dir}\'\n')
        elif 'ninstances =' in line:
            newlines.append(f'ninstances = {ninstances}\n')
        else:
            newlines.append(line)
    with open(filepath, 'w') as fl:
        fl.writelines(newlines)


    for T in temps:
        temp_dir = f'{root_dir}T{T}K/'
        os.system(f'mkdir -p {temp_dir}')
        os.system(f'ln -s -f {scripts_dir}* {temp_dir}')


In [1]:
import my_utils
import os
print(my_utils.__file__)

None
